In [7]:
import json
import pandas as pd
 
import requests
from random import choice
from bs4 import BeautifulSoup

In [1]:
_user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
]
 
 
class InstagramScraper:
 
    def __init__(self, user_agents=None, proxy=None):
        self.user_agents = user_agents
        self.proxy = proxy
 
    def __random_agent(self):
        if self.user_agents and isinstance(self.user_agents, list):
            return choice(self.user_agents)
        return choice(_user_agents)
 
    def __request_url(self, url):
        try:
            response = requests.get(url, headers={'User-Agent': self.__random_agent()}, proxies={'http': self.proxy,
                                                                                                 'https': self.proxy})
            response.raise_for_status()
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from Instagram')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return response.text
 
    @staticmethod
    def extract_json_data(html):
        soup = BeautifulSoup(html, 'html.parser')
        body = soup.find('body')
        script_tag = body.find('script')
        raw_string = script_tag.text.strip().replace('window._sharedData =', '').replace(';', '')
        return json.loads(raw_string)
 
    def profile_page_metrics(self, profile_url):
        results = {}
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']
        except Exception as e:
            raise e
        else:
            for key, value in metrics.items():
                if key != 'edge_owner_to_timeline_media':
                    if value and isinstance(value, dict):
                        value = value['count']
                        results[key] = value
                    elif value:
                        results[key] = value
        return results
 
    def profile_page_recent_posts(self, profile_url):
        results = []
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']["edges"]
        except Exception as e:
            raise e
        else:
            for node in metrics:
                node = node.get('node')
                if node and isinstance(node, dict):
                    results.append(node)
        return results
        

Montando base de indicativos ofertas através do instagram. Estas palavras serão buscadas no texto das postagens das lojas. Se forem encontradas, são fortes indícios de que a postagem é sobre desconto.

In [21]:
palavras_indicativo_oferta = ['oferta', 'promoç', 'desconto', 'liquid', 'off', '%']

In [22]:
k = InstagramScraper()

lojas = pd.read_csv('lojas.csv')
lojas_com_instagram = lojas[pd.notnull(lojas['REDE SOCIAL'])]
tem_oferta = []
for i in range(0, len(lojas_com_instagram)):
    data = k.profile_page_recent_posts(lojas_com_instagram.iloc[[i]]['REDE SOCIAL'].values[0])
    tem_oferta.append(False)
    for postagem in data:
        texto = postagem['edge_media_to_caption']['edges'][0]['node']['text']
        for palavra in palavras_indicativo_oferta:
            if palavra in texto:
                tem_oferta[-1] = True
                break
        if tem_oferta[-1]:
            break

In [25]:
lojas_com_instagram['tem_oferta'] = tem_oferta

/home/marcelo/cement_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
lojas_com_instagram[lojas_com_instagram['tem_oferta']].to_csv('lojas_oferta.csv', index=False)